# Census Data
Collects ACS data from census.gov API and merges it into one table.

If you want to collect your own census data, you'll need to regiester for an [API key](https://api.census.gov/data/key_signup.html), and assign it as the environment variable `CENSUS_API_KEY`. 

This is not necessary, as all outputs are calcualted and saved in this repository already.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import requests
import glob as glob
import gzip
from multiprocessing import Pool

from tqdm import tqdm
import numpy as np
import censusdata
import pandas as pd
import geopandas as gpd

## Download ACS data

In [4]:
# ouputs
data_dir = '../data/input/census/acs5'
os.makedirs(data_dir, exist_ok=True)

# inputs
fn_income = '../data/input/census/state2income.json'
state2income = json.load(open(fn_income, 'r'))

input_files = glob.glob('../data/input/isp/*/*/*.geojson.gz')
fips = set([_.split('/')[-1].split('.geojson')[0][:5] for _ in input_files])
fips = set([_ for _ in fips if _ != 'spotc'])
len(fips)

# params
recalculate = False
API_KEY = os.environ.get('CENSUS_API_KEY')
if not API_KEY:
    print("Generate census API key and set as env variable `CENSUS_API_KEY`. See: https://api.census.gov/data/key_signup.html")

95

In [5]:
# the three ACS tables and columns we're going to get.
acs_tables = [
    {
        "display_name": "race_ethnicity",
        "table_name": "B03002",
        "url": "https://api.census.gov/data/2018/acs/acs5/groups/B03002.html",
        "columns": {
            "block group": "block_group",
            "B03002_001E": "race_total_estimate",
            "B03002_003E": "race_white_alone",
            "B03002_004E": "race_black_alone",
            "B03002_005E": "race_aindian_alone",
            "B03002_006E": "race_asian_alone",
            "B03002_007E": "race_pacific_islander_native_hawaiian_alone",
            "B03002_008E": "race_some_other_alone",
            "B03002_009E": "race_two_or_more_alone",
            "B03002_012E": "race_latino_alone",
        }
    },
    {
        "display_name": "household_income_2",
        "table_name": 'B19013',
        "url": "https://api.census.gov/data/2018/acs/acs5/groups/B19013.html",
        "columns": {
            "block group": "block_group",
            "B19013_001E": "median_household_income",
        }
    },
    {
        "display_name": "internet_subscription",
        "table_name": 'B28002',
        "url": "https://api.census.gov/data/2018/acs/acs5/groups/B28002.html",
        "columns": {
            "block group": "block_group",
            "B28002_001E": "internet_total_estimate",
            "B28002_002E": "internet_subscriptions_any",
            "B28002_004E": "internet_broadband",
            "B28002_005E": "internet_mobile",
            "B28002_006E": "internet_mobile_only",
            "B28002_013E": "internet_no_access",
        }
    }
]

In [6]:
def make_acs_request(year, column, geography, state, county, api_key, debug=False):
    """
    Formats the API call and make the reuqest
    """
    county = str(county).zfill(3)
    url = (f"https://api.census.gov/data/{year}/acs/acs5?get={column}&"
           f"for={geography}:*&in=state:{state}%20county:{county}&key={api_key}")
    if debug:
        print(url)
    return requests.get(url)

def download_acs(column: str= "B03002_001E",
                 geography: str = "block group",
                 year: int= 2019, 
                 state: int= 55,
                 county: [int, str]= 1,
                 data_dir: str= 'data/',
                 api_key: str= None,
                 debug: bool= False) -> dict:
    """
    geography can also be "tract",
    column is the ACS "table name"
    """
    # check if the file exists...
    table = column.split('_')[0]
    geography_ = geography.replace(' ', '_')
    fn_out = f"{data_dir}/{geography_}/{year}/{state}/{county}/{table}/{column}.csv.gz"
    if os.path.exists(fn_out):
        return 1
    os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    
    # make the request
    resp = make_acs_request(year, column, geography, state, county, api_key, debug)
    
    # validate the response and save it
    if resp.status_code != 200:
        pd.DataFrame([]).to_csv(fn_out, index=False, compression='gzip')
        return 0
    _data = resp.json()
    _df = pd.DataFrame(_data[1:], columns=_data[0])
    _df.to_csv(fn_out, index=False, compression='gzip')
    return 1

To speed things up, we're going to use multiprocessing to collect these files.
First, we must create a list of arguments for each API request.

In [7]:
year = 2019
debug = False
args = []
for fip in fips:
    state = fip[:2]
    county = fip[2:]
    for geography in ['block group']:
        for table in acs_tables:
            table_name = table["display_name"]
            columns = [c for c in table["columns"].keys() if c != "block group"]
            for column in columns:
                args.append([column, geography, year, state, county, data_dir, API_KEY, debug])

f"We must make {len(args)} API calls."

'We must make 1520 API calls.'

In [8]:
# use multiprocessing to make the requests...
def collect_data():
    with Pool(64) as pool:
        pool.starmap(download_acs, tqdm(args, total=len(args)))
collect_data()

100%|██████████| 1520/1520 [00:00<00:00, 93822.64it/s]


## Data processing
With the ACS data collected, we're now going to calculate the racial demographics, income levels, and broadband pentration for each block group.

In [9]:
# outputs
fn_out_acs = '../data/intermediary/census/aggregated_tables.csv.gz'
os.makedirs(os.path.dirname(fn_out_acs), exist_ok=True)

In [10]:
def get_relevant_block_groups(verbose=False):
    """Get the block groups in the data we have"""
    files_input = glob.glob("../data/input/isp/*/*/*.geojson.gz")    
    block_groups = set([f.split('/')[-1].split('.geojson')[0] for f in files_input])
    block_groups = set([bg for bg in block_groups if len(bg) == 12])
    if verbose:
        print(f"Found {len(block_groups)} block groups.")
    
    return block_groups

In [11]:
def get_geoid(row):
    """
    Explictly cast block group-level geoid
    """
    state = int(row['state'])
    county = int(row['county'])
    tract = int(row['tract'])
    block = int(row['block_group'])
    
    return f"{state:02}{county:03}{tract:06}{block:01}"

def percent_non_white(row):
    """
    Percentage of area that are not non-Hispanic white
    """
    total = row['race_total_estimate']
    white = row['race_white_alone']
    try:
        perc_non_white =  (total - white) / total
        return max(0, min(perc_non_white, 1))
    except ZeroDivisionError:
        return None
    
def income_level(row):
    """
    Calculate median household income compared to city median income.
    We set None for skip values set by the census bureau.
    """
    state = row['state']
    if row['median_household_income'] == -666666666:
        return None
    median_income = state2income.get(str(state), {})
    if median_income:
        return row['median_household_income'] / median_income
    return None

def dollars_diff_median(row):
    """
    Get dollars below median city income.
    """
    state = row['state']
    if row['median_household_income'] == -666666666:
        return None
    median_income = state2income.get(str(state), {})
    if median_income:
        return median_income - row['median_household_income']
    return None

In [12]:
if not os.path.exists(fn_out_acs) or recalculate:
    # This merges and concats each ACS table we collected.
    geography = "block group"
    df = pd.DataFrame([])
    for table in acs_tables:
        col2rename = table['columns']
        table_name = table['table_name']
        cols = [c for c in col2rename.keys() if c != "block group"]
        for column in tqdm(cols):
            files = glob.glob(f"../data/input/census/acs5/{geography.replace(' ', '_')}/{year}/*/*/{table_name}/{column}.csv.gz")
            data = []
            for fn in files:
                tp = pd.read_csv(fn, compression='gzip')
                if tp.empty:
                    print(fn)
                data.extend(tp.to_dict("records"))
            tmp = pd.DataFrame(data)
            tmp.columns = [col2rename.get(c, c) for c in tmp.columns]
            if df.empty:
                df = tmp
            else:
                df = df.merge(tmp, on= ["state", "county", "tract", "block_group"], how='outer')

    # process the data
    df['race_perc_non_white'] = df.apply(percent_non_white, axis=1)
    df['internet_perc_broadband'] = df['internet_broadband'] / df['internet_total_estimate']
    df['geoid'] = df.apply(get_geoid, axis=1)
    
    # For whatever reason, calling these functions can return None.
    # check how many null values they produce, and run again if there are a lot of nulls.
    df['income_dollars_below_median'] = df.apply(dollars_diff_median, axis=1)
    df['income_lmi'] = df.apply(income_level, axis=1)
    
    df.to_csv(fn_out_acs, index=False, compression='gzip')
else:
    df = pd.read_csv(fn_out_acs, compression='gzip')

100%|██████████| 6/6 [00:03<00:00,  1.85it/s]
<ipython-input-11-d2142ac20b47>:19: RuntimeWarning: invalid value encountered in long_scalars
  perc_non_white =  (total - white) / total


In [13]:
df.groupby('state').income_lmi.mean()

state
1     1.140849
4     1.178269
5     1.058973
6     1.195220
8     1.140803
9     2.452951
10    1.690188
11    1.106058
12    1.055303
13    1.228180
16    1.003993
17    1.171541
18    1.029438
19    1.247708
20    1.056337
21    1.154028
22    1.265439
24    1.760873
25    1.240961
26    1.579395
27    1.323721
28    1.215171
29    1.130529
30    1.062151
31    1.172908
32    1.076064
34    2.454742
35    1.085823
36    1.189430
37    1.162922
38    1.135345
39    1.794647
40    1.061249
41    1.128865
42    1.219454
44    1.320368
45    0.927050
46    1.112078
47    1.282997
48    1.342976
49    1.258911
51    0.953377
53    1.088911
54    1.052705
55    1.333893
56    0.945529
Name: income_lmi, dtype: float64

## Processing raw shape file for maps

[TIGER](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html) shape files were downloaded and stored in one place (`../data/input/census/shape/raw`).

In [14]:
# ouputs
fn_out_shp = '../data/intermediary/census/2019_acs_5_shapes.geojson.gz'
files_geojson = '../data/input/census/shape/preprocessed/*.geojson'

# inputs
files_shape = glob.glob('../data/input/census/shape/raw/tl_2019_*_bg.zip')
len(files_shape)

46

We're going to filter out block groups that aren't in our investigation, merge ACS demographic data, and combine the shapes into one file.

the zip files need to be converted into geoJSON (via geopandas `gpd`) to be useful to us.

In [15]:
# convert zip files to geoJSON
for fn in tqdm(files_shape):
    fn_out = fn.replace('/raw/', '/preprocessed/').replace('.zip', '.geojson')
    if os.path.exists(fn_out):
        continue
    shp = gpd.read_file('zip:///' + fn)
    shp.to_file(fn_out, driver='GeoJSON')

100%|██████████| 46/46 [00:00<00:00, 65049.89it/s]


In [16]:
if not os.path.exists(fn_out_shp) or recalculate:
    df = pd.read_csv(fn_out_acs, compression='gzip')
    df.geoid = df.geoid.apply(lambda x: f"{x:012d}")
    df['income_level'] = pd.cut(
        df['income_lmi'],
        bins=[-1e10, .5, .8, 1.2, 1e10],
        labels=['Low', 'Moderate', 'Middle', 'Upper Income'],
        right=False
    )
    
    block_groups = get_relevant_block_groups(verbose=True)    
    geoid2meta = {row['geoid']: {
        'race_perc_non_white' : row['race_perc_non_white'], 
        'income_level' : row['income_level'],
        'median_household_income' : row['median_household_income']
    }  for i, row in df[df.geoid.isin(block_groups)].iterrows()}
    len(geoid2meta)
    
    files_geojson = glob.glob(files_geojson)
    geography = "block group"
    to_save = []
    for fn_geojson in tqdm(files_geojson):
        with open(fn_geojson , 'r') as f:
            geojson = json.load(f)
            for i, feature in enumerate(geojson['features']):
                featureProperties = feature['properties']
                if geography == "block group":
                    geoid = featureProperties['GEOID']
                featureData = geoid2meta.get(geoid, {})
                for key in featureData.keys():
                    featureProperties[key] = featureData[key]
                if featureData:
                    to_save.append(feature)
    
    # save geoJSON with acs data merged in
    geojson['features'] = to_save
    with gzip.open(fn_out_shp, 'wt') as f:
        f.write(json.dumps(geojson).replace('NaN', '""'))

Found 28759 block groups.


100%|██████████| 46/46 [01:06<00:00,  1.44s/it]


## population density & number of competing ISPs
Final tweaks to the ACS data to be used for analysis.

We reference TIGER shape files to calculate population density using the the area of land for each census block group and the estimated population. We aldo merge the number of competing ISPs for each block group according to the FCC's Form 477.

In [17]:
# ouputs
fn_out_features = '../data/intermediary/census/aggregated_tables_plus_features.csv.gz'
fn_providers = '../data/intermediary/fcc/bg_providers.csv'

In [18]:
# inputs
fn_form_477 = '../data/input/fcc/fbd_us_with_satellite_dec2020_v1.csv.gz'

In [19]:
# get number of competitors from FCC Form 477.
if not os.path.exists(fn_providers) or recalculate:
    # Filter for block groups in our invetigation.
    block_groups = get_relevant_block_groups(verbose=True)
    data = []
    for _df in pd.read_csv(fn_form_477, 
                           compressiom= 'gzip',
                           encoding= 'unicode_escape', 
                           chunksize= 50000, 
                           dtype= {'BlockCode' : str}):
        _df['block_group'] = _df['BlockCode'].apply(lambda x: x[:12])
        data.extend(_df[_df['block_group'].isin(block_groups)]
                                          .to_dict(orient='records'))
    df_bg = pd.DataFrame(data)
    
    (df_bg[
        (df_bg.Consumer == 1) &
        (~df_bg.TechCode.isin([60,70]))   
    ].groupby('block_group')['ProviderName']
     .nunique().to_frame('n_providers')
     .to_csv(fn_providers)
    )

In [20]:
def calculate_persons_per_sq_mile(row):
    """
    See formula referenced in these two sources:
    https://acsdatacommunity.prb.org/discussion-forum/f/forum/585/moe-for-population-density
    https://www.census.gov/quickfacts/fact/note/US/LND110210
    1,000,000 * POP / ALAND
    """
    if row['ALAND'] == 0:
        return None
    sq_miles = row['ALAND'] / 1000000
    persons = row['race_total_estimate']
    return persons / sq_miles

In [21]:
if os.path.exists(fn_out_features) or recalculate:
    df = pd.read_csv(fn_out_acs, compression='gzip', dtype={'geoid': str})
    df_shp = pd.DataFrame([row['properties'] for row in 
                           json.load(gzip.open(fn_out_shp, 'r'))['features']])
    df_providers = pd.read_csv(fn_providers, dtype={'block_group': str})
    
    df = df.merge(df_shp[['GEOID', 'ALAND']], 
                  how='left', left_on='geoid', right_on='GEOID', 
                  suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
    df['ppl_per_sq_mile'] = df.apply(calculate_persons_per_sq_mile, axis=1)

    df = df.merge(df_providers, 
                  how='left', left_on='GEOID', right_on='block_group', 
                  suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
    
    df.to_csv(fn_out_features, index=False, compression='gzip')

In [22]:
df

,race_total_estimate,state,county,tract,block_group,race_white_alone,race_black_alone,race_aindian_alone,race_asian_alone,race_pacific_islander_native_hawaiian_alone,...,internet_no_access,race_perc_non_white,internet_perc_broadband,geoid,income_dollars_below_median,income_lmi,GEOID,ALAND,ppl_per_sq_mile,n_providers
0,3594,28,45,30602,2,3389,0,0,0,0,...,425,0.057040,0.610870,280450306022,-6865.0,1.171351,NaN,NaN,NaN,NaN
1,1114,28,45,30602,1,1008,2,31,0,0,...,42,0.095153,0.911017,280450306021,-17749.0,1.443016,280450306021,30486858.0,36.540335,3.0
2,0,28,45,990000,0,0,0,0,0,0,...,0,0.000000,NaN,280459900000,NaN,NaN,NaN,NaN,NaN,NaN
3,667,28,45,30602,3,579,0,0,0,0,...,34,0.131934,0.883162,280450306023,-18302.0,1.456819,280450306023,359392211.0,1.855911,3.0
4,604,28,45,30300,4,570,0,0,0,0,...,96,0.056291,0.660777,280450303004,1403.0,0.964981,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52497,1425,47,157,20644,3,447,931,0,0,0,...,291,0.686316,0.652745,471570206443,18787.0,0.551237,471570206443,1833522.0,777.192747,2.0
52498,1528,47,157,20643,1,1318,102,0,56,0,...,0,0.137435,0.971223,471570206431,-60542.0,2.446159,471570206431,943336.0,1619.783407,2.0
52499,759,47,157,21351,4,682,0,0,77,0,...,26,0.101449,0.917722,471570213514,-40059.0,1.956884,471570213514,762270.0,995.710181,2.0
52500,1231,47,157,21352,4,1132,0,0,26,0,...,0,0.080422,0.800875,471570213524,-85147.0,3.033895,471570213524,1413116.0,871.124522,2.0
